# Introduction

This notebook trains an NLP model with PySpark and Spark NLP. Three outputs will be produced:
1. Word cloud
2. Sentiment scoring
3. Topic modelling

The pretrained models are downloaded manually and placed in `models/pretrained`:
* [analyze_sentimentdl_use_twitter](https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/models/analyze_sentimentdl_use_twitter_en_2.7.1_2.4_1610993470852.ziphttps://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/models/analyze_sentimentdl_use_twitter_en_2.7.1_2.4_1610993470852.zip) (935MB)

# Package installations

# Imports

In [4]:
import pandas as pd
from pathlib import Path
from pyspark.sql import (
    SparkSession,
    functions as F
)
import sparknlp
from pyspark.ml import (
    Pipeline
)
from sparknlp.pretrained import PretrainedPipeline
from wordcloud import (
    WordCloud, 
    STOPWORDS, 
    ImageColorGenerator
)

In [5]:
spark = sparknlp.start()
print(f'sparknlp.version(): {sparknlp.version()}')
print(f'spark.version: {spark.version}')

sparknlp.version(): 3.0.3
spark.version: 3.1.1


# Set up directories

In [6]:
project_dir = Path.cwd().parent
models_dir = project_dir / 'models'
pretrained_models_dir = models_dir / 'pretrained'
data_dir = project_dir / 'data'
raw_data_dir = data_dir / 'raw'

# Pretrained pipelines

In [8]:
??PretrainedPipeline

Init signature:
PretrainedPipeline(
    name,
    lang='en',
    remote_loc=None,
    parse_embeddings=False,
    disk_location=None,
)
Docstring:      <no docstring>
Source:        
class PretrainedPipeline:

    def __init__(self, name, lang='en', remote_loc=None, parse_embeddings=False, disk_location=None):
        if not disk_location:
            self.model = ResourceDownloader().downloadPipeline(name, lang, remote_loc)
        else:
            self.model = PipelineModel.load(disk_location)
        self.light_model = LightPipeline(self.model, parse_embeddings)

    @staticmethod
    def from_disk(path, parse_embeddings=False):
        return PretrainedPipeline(None, None, None, parse_embeddings, path)

    def annotate(self, target, column=None):
        if type(target) is DataFrame:
            if not column:
                raise Exception("annotate() column arg needed when targeting a DataFrame")
            return self.model.transform(target.withColumnRenamed(column, "text"))

In [15]:
model_name = 'analyze_sentimentdl_use_twitter'
disk_location = pretrained_models_dir / model_name

pipeline = PretrainedPipeline(
    name=model_name,
    disk_location=disk_location.as_posix()
)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 22.0 failed 1 times, most recent failure: Lost task 0.0 in stage 22.0 (TID 69) (9aa2c961ca1b executor driver): org.apache.hadoop.fs.ChecksumException: Checksum error: file:/home/jovyan/work/models/pretrained/analyze_sentimentdl_use_twitter/metadata/part-00000 at 0 exp: 145320018 got: -117765660
	at org.apache.hadoop.fs.FSInputChecker.verifySums(FSInputChecker.java:347)
	at org.apache.hadoop.fs.FSInputChecker.readChecksumChunk(FSInputChecker.java:303)
	at org.apache.hadoop.fs.FSInputChecker.fill(FSInputChecker.java:215)
	at org.apache.hadoop.fs.FSInputChecker.read1(FSInputChecker.java:256)
	at org.apache.hadoop.fs.FSInputChecker.read(FSInputChecker.java:197)
	at java.io.DataInputStream.read(DataInputStream.java:149)
	at org.apache.hadoop.mapreduce.lib.input.UncompressedSplitLineReader.fillBuffer(UncompressedSp

# Load parquet

In [13]:
path = raw_data_dir / 'twitter_flat.parquet'
# `spark.read.parquet` can't infer schema for some reason
# so load into pandas df first
pdf = pd.read_parquet(path)
df = spark.createDataFrame(pdf)

In [22]:
df.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- is_quote_status: boolean (nullable = true)
 |-- in_reply_to_user_id: double (nullable = true)
 |-- user_id: long (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_created_at: string (nullable = true)
 |-- extended_full_text: string (nullable = true)
 |-- retweeted_status_id: double (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- is_retweet: boolean (nullable = true)



In [19]:
df.show(5)

+--------------------+-------------------+--------------------+---------------+--------------------+-------------------+--------------------+------------------+--------------------+--------------------+--------------------+-------------+--------------+-----------+-----------+----------+
|          created_at|                 id|                text|is_quote_status| in_reply_to_user_id|            user_id|user_followers_count|user_friends_count|     user_created_at|  extended_full_text| retweeted_status_id|retweet_count|favorite_count|quote_count|reply_count|is_retweet|
+--------------------+-------------------+--------------------+---------------+--------------------+-------------------+--------------------+------------------+--------------------+--------------------+--------------------+-------------+--------------+-----------+-----------+----------+
|Sun May 30 12:35:...|1398981377031163908|Hey @news_of_bsc ...|          false|                 NaN|1392022650529013764|                

In [42]:
text = df.first().asDict()['extended_full_text']
text

'Hey @news_of_bsc you should interview 🚀the new project @twinFinance ⏰Launching on June 15th. They are having an 🪂airdrop right now #airdrop #twinfinance #BSC #Binance #ICO #token https://t.co/pLkznpqFsZ'

In [43]:
pipeline.annotate(text)

{'document': ['Hey @news_of_bsc you should interview 🚀the new project @twinFinance ⏰Launching on June 15th. They are having an 🪂airdrop right now #airdrop #twinfinance #BSC #Binance #ICO #token https://t.co/pLkznpqFsZ'],
 'sentence_embeddings': ['Hey @news_of_bsc you should interview 🚀the new project @twinFinance ⏰Launching on June 15th. They are having an 🪂airdrop right now #airdrop #twinfinance #BSC #Binance #ICO #token https://t.co/pLkznpqFsZ'],
 'sentiment': ['positive']}